In [377]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os


In [378]:
directory = '/Users/goikhmanb/Desktop/2025-12-12_Tunnel_MS2P_3'
files = os.listdir(directory)

delimiter = '\t'  # Change to ',' if your files are comma-separated

In [379]:
files

['S8_M6_1.dat',
 'S8_M6_2.dat',
 'S8_M8_man_neg_bias.dat',
 'S8_M8_man_neg_bias_1.dat',
 'S8_M3_1.dat',
 'S8_M3.dat',
 'S8_M8_man.dat',
 'S8_M4.dat',
 'S8_M6.dat',
 'S8_M8_aut_3.dat',
 'S8_M8_aut_4.dat',
 'S8_M8_aut_6.dat']

In [380]:
filenames = [  'S8_M8_aut_3.dat']
ttl = filenames[0].split('_')[0] + '_' + filenames[0].split('_')[1]
ttl

'S8_M8'

In [381]:
def merger(filenames):
    if len(filenames) == 1:
        df = pd.read_csv(os.path.join(directory, filenames[0]), delimiter=delimiter)
        return df
    df_list = []
    for filename in filenames:
        df = pd.read_csv(os.path.join(directory, filename), delimiter=delimiter)
        df_list.append(df)
    df =  pd.concat(df_list, axis = 0)
    df.index = range(len(df))

    return df

In [382]:

data = merger(filenames = filenames)
#data = pd.read_csv(os.path.join(directory, filename), delimiter=delimiter)


In [383]:
len(data['Vsample_V'].unique())

101

In [384]:

mean_per_v = (
    data
    .groupby('Vsample_V')['Vsample_I']
    .mean()
)

data['Vsample_I_mean'] = data['Vsample_V'].map(mean_per_v)

# keep mean only on the first occurrence of each V
data.loc[data.duplicated('Vsample_V'), 'Vsample_I_mean'] = float('nan')



In [385]:
data['Vsample_I_mean_rolling'] = data['Vsample_I_mean'].rolling(window=10, center=True).mean()

data 

,Vsample_V,Vsample_I,Vsample_I_mean,Vsample_I_mean_rolling
0,0.000,-1.796984e-11,-2.218140e-11,NaN
1,0.035,-2.430603e-11,-2.306143e-11,NaN
2,0.070,-3.164796e-11,-2.243284e-11,NaN
3,0.105,-2.123851e-11,-2.101223e-11,NaN
4,0.140,-1.444973e-11,-2.014478e-11,NaN
...,...,...,...,...
399,0.140,-1.952875e-11,NaN,NaN
400,0.105,-2.063508e-11,NaN,NaN
401,0.070,-2.154025e-11,NaN,NaN
402,0.035,-2.249570e-11,NaN,NaN


In [387]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data['Vsample_V'],
        y=data['Vsample_I'],
        mode='lines',
        name='raw',
        marker=dict(
            color='rgba(0, 0, 0, 0.2)'
        )
    )
)

for file in filenames:
    label = file
    df = pd.read_csv(os.path.join(directory, file), delimiter=delimiter)
    fig.add_trace(
        go.Scatter(
            x=df['Vsample_V'],
            y=df['Vsample_I'],
            mode='lines',
            name=label,
            line=dict(
                width=1
            )
        )
    )

# mean I(V) (line)
fig.add_trace(
    go.Scatter(
        x=data['Vsample_V'],
        y=data['Vsample_I_mean'],
        mode='lines',
        name='mean',
        line=dict(
            color='rgba(0, 0, 255, 0.9)', 
            width=3
        )
    )
)
# smoothed mean I(V) (line)
fig.add_trace(
    go.Scatter(
        x=data['Vsample_V'],
        y=data['Vsample_I_mean_rolling'],
        mode='lines', 
        name='smoothed',
        line=dict(
            color='rgba(255, 0, 0, 0.9)',
            width=3
        )
    )
)

fig.update_layout(
    title = f'{filenames}',
    xaxis_title='Vsample_V',
    yaxis_title='Vsample_I', 
    
)


fig.show()


In [388]:
filesave = True

if filesave:
    data.to_csv(os.path.join(directory, f'{ttl}_prcd.csv'), index=False)
    [os.remove(os.path.join(directory, filename)) for filename in filenames]